1. 安装mindspore2.4.1，安装指南详见：[MindSpore安装](https://www.mindspore.cn/install)
2. 安装MindNLP及相关依赖，MindNLP官方仓详见：[MindNLP](https://github.com/mindspore-lab/mindnlp)

In [1]:
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.4.1/MindSpore/unified/aarch64/mindspore-2.4.1-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.5/335.5 MB 8.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.4/442.4 kB 5.0 MB/s eta 0:00:00a 0:00:01
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: mindspore
    Found existing installation: mindspore 2.3.0
    Uninstalling mindspore-2.3.0:
      Successfully uninstalled mindspore-2.3.0


In [2]:
!pip install mindnlp

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 22.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 43.2 MB/s eta 0:00:00
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.0.1
    Uninstalling Pillow-9.0.1:
      Successfully uninstalled Pillow-9.0.1
ERR

In [2]:
!pip uninstall mindformers -y

Found existing installation: mindformers 0.8.0
Uninstalling mindformers-0.8.0:
  Successfully uninstalled mindformers-0.8.0


***注：执行如上命令完成安装后，请点击上方的restart kernel图标重启kernel，再进行实验***

### 数据集加载与处理

1. 数据集加载

    本次实验使用的是nlpcc2017摘要数据，内容为新闻正文及其摘要，总计50000个样本。

In [3]:
from mindnlp.utils import http_get

# download dataset
url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path = http_get(url, './')

In [4]:
from mindspore.dataset import TextFileDataset

# load dataset
dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

50000

**本案例默认在GPU P100上运行，因中文文本，tokenizer使用的是bert tokenizer而非gpt tokenizer等原因，全量数据训练1个epoch的时间约为80分钟。**

**为节约时间，我们选取了数据集中很小的一个子集（500条数据）来演示gpt2的微调和推理全流程，但由于数据量不足，会导致模型效果较差。**

In [5]:
# split into training and testing dataset
mini_dataset, _ = dataset.split([0.01, 0.99], randomize=False)
train_dataset, test_dataset = mini_dataset.split([0.9, 0.1], randomize=False)

2. 数据预处理

    原始数据格式：
    ```text
    article: [CLS] article_context [SEP]
    summary: [CLS] summary_context [SEP]
    ```
    预处理后的数据格式：

    ```text
    [CLS] article_context [SEP] summary_context [SEP]
    ```

In [6]:
import json
import numpy as np

# preprocess dataset
def process_dataset(dataset, tokenizer, batch_size=4, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        # tokenization
        # pad to max_seq_length, only truncate the article
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['input_ids']
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    # change column names to input_ids and labels for the following training
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

因GPT2无中文的tokenizer，我们使用BertTokenizer替代。

In [7]:
from mindnlp.transformers import BertTokenizer

# We use BertTokenizer for tokenizing chinese context.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
len(tokenizer)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


21128

In [8]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=4, max_seq_len=300)

In [9]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[4, 300], dtype=Int64, value=
 [[ 101, 1724, 3862 ... 3299, 2419,  102],
  [ 101,  704, 3173 ... 3341, 3975,  102],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]]),
 Tensor(shape=[4, 300], dtype=Int64, value=
 [[ 101, 1724, 3862 ... 3299, 2419,  102],
  [ 101,  704, 3173 ... 3341, 3975,  102],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]])]

### 模型构建

构建GPT2ForSummarization模型，注意***shift right***的操作。

In [10]:
from mindnlp.core.nn import functional as F
from mindnlp.transformers import GPT2LMHeadModel

class GPT2ForSummarization(GPT2LMHeadModel):
    def forward(
        self,
        input_ids = None,
        attention_mask = None,
        labels = None,
    ):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        shift_logits = outputs.logits[..., :-1, :]
        shift_labels = labels[..., 1:]
        # Flatten the tokens
        loss = F.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
        return (loss,)

### 模型训练

In [11]:
num_epochs = 1
warmup_steps = 100
learning_rate = 1.5e-4

max_grad_norm = 1.0
num_training_steps = num_epochs * train_dataset.get_dataset_size()

In [12]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=len(tokenizer), bos_token_id=tokenizer.cls_token_id, eos_token_id=tokenizer.pad_token_id)
model = GPT2ForSummarization(config)

GPT2LMHeadModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
[WARNING] DEVICE(3028,ffffa994f010,python):2024-12-23-17:56:18.694.422 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:188] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.rc2.2


In [13]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

number of model parameters: 102068736


In [14]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="./output/gpt2_summarization",
    save_steps=train_dataset.get_dataset_size(),
    save_total_limit=3,
    logging_steps=1000,
    max_steps=num_training_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    warmup_steps=warmup_steps
    
)

from mindnlp.engine import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [15]:
trainer.train()

  0%|          | 0/113 [00:00<?, ?it/s]

  1%|          | 1/113 [00:36<1:08:25, 36.65s/it]

100%|██████████| 113/113 [02:16<00:00,  2.11it/s]

100%|██████████| 113/113 [02:28<00:00,  1.32s/it]

{'train_runtime': 148.8578, 'train_samples_per_second': 6.073, 'train_steps_per_second': 0.759, 'train_loss': 7.764800215189436, 'epoch': 1.0}


TrainOutput(global_step=113, training_loss=7.764800215189436, metrics={'train_runtime': 148.8578, 'train_samples_per_second': 6.073, 'train_steps_per_second': 0.759, 'train_loss': 7.764800215189436, 'epoch': 1.0})

In [16]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized['input_ids']

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [17]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [18]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

[array([[ 101,  126, 3299, 8110, 3189, 3241, 8024, 3633,  966, 2844, 1894,
        5688,  511, 3727,  704, 2356,  704, 2552, 1278, 7368, 6117, 3890,
        7599, 3969, 1048, 4554, 4906, 2844, 1894, 2207, 6145, 8024, 3633,
        1762, 3146, 4415, 2843, 3131, 2642, 5442, 4500, 4638, 4289, 1501,
        8024, 1762, 3690, 3187, 7344, 1906, 4638, 2658, 1105,  678, 8024,
        6158,  671, 4511, 2094,  697, 3613, 3666, 2802,  511,  113, 1290,
        1555, 2845,  126, 3299, 8115, 3189,  100, 4276, 3295, 2845, 6887,
         114,  126, 3299, 8130, 3189,  677, 1286, 8024, 1290, 1555, 2845,
        6381, 5442,  794, 1062, 2128, 3727, 1378, 6356, 3175,  749, 6237,
        1168, 8024, 2802,  782, 4511, 2094, 1426, 3378, 2347, 6158, 6121,
        3124, 2872, 4522,  511, 2945, 6356, 3175,  792, 5305, 8024,  126,
        3299, 8110, 3189, 2496, 3241, 8024, 1728, 1426, 3378, 5326, 3678,
        1728, 2843, 3131, 3187, 3126, 3647,  767, 8024, 6628, 1168, 1278,
        7368, 4638, 1426, 3378, 2658,

In [19]:
model = GPT2LMHeadModel.from_pretrained('./output/gpt2_summarization/checkpoint-113/model.safetensors', config=config)

由于训练数据量少，epochs数少且tokenizer并未使用gpt tokenizer等因素，模型推理效果会较差。

In [20]:
model.set_train(False)
model.config.eos_token_id = model.config.sep_token_id
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=50, num_beams=5, no_repeat_ngram_size=2)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 1:
        break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[CLS] 5 月 12 日 晚 ， 正 值 护 士 节 。 汉 中 市 中 心 医 院 血 液 风 湿 免 疫 科 护 士 小 袁 ， 正 在 整 理 抢 救 患 者 用 的 物 品 ， 在 毫 无 防 备 的 情 况 下 ， 被 一 男 子 两 次 殴 打 。 ( 华 商 报 5 月 15 日 [UNK] 版 曾 报 道 ) 5 月 22 日 上 午 ， 华 商 报 记 者 从 公 安 汉 台 警 方 了 解 到 ， 打 人 男 子 吴 某 已 被 行 政 拘 留 。 据 警 方 介 绍 ， 5 月 12 日 当 晚 ， 因 吴 某 继 母 因 抢 救 无 效 死 亡 ， 赶 到 医 院 的 吴 某 情 绪 激 动 ， 将 护 士 站 护 士 小 袁 打 伤 ， 导 致 小 袁 头 部 外 伤 ， 全 身 多 处 软 组 织 挫 伤 。 医 院 报 警 后 ， 吴 某 逃 离 现 场 。 事 件 发 生 后 ， 民 警 第 一 时 间 展 开 调 查 ， 锁 定 嫌 疑 人 吴 某 。 随 后 ， 吴 某 被 警 方 传 唤 ， 对 违 法 事 实 供 认 不 讳 。 目 前 ， 吴 某 因 涉 嫌 寻 衅 滋 事 ， 被 警 方 行 政 拘 留 15 日 。 华 商 报 记 者 从 汉 中 市 中 心 医 院 了 解 到 ， 小 袁 已 出 院 ， 回 到 自 己 的 工 作 岗 位 。 华 商 记 者 < [UNK] > 王 亮 编 辑 ： 华 商 报 供 稿 [SEP] [UNK] 。 [SEP] 一 个 月 ， 中 国 人 ， 一 一 人 员 ， 这 ， 他 中 。 图 [SEP] 中 ， [UNK] ， 不 在 一 名 ， 其 中 的 一 子 ， 并 人 在 人 人 的 [UNK] 的 ， 称 ， ，
